In [27]:
import qstock as qs
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from rqdatac import *
import rqdatac
import rqdatac_fund
import datetime
from tqdm import tqdm
from dateutil.relativedelta import relativedelta
rqdatac.init()
from sklearn.model_selection import train_test_split

/Users/songhanhu/opt/anaconda3/lib/python3.9/site-packages/rqdatac/client.py:202: UserWarning: rqdatac is already inited. Settings will be changed.
  warnings.warn("rqdatac is already inited. Settings will be changed.", stacklevel=0)
/Users/songhanhu/opt/anaconda3/lib/python3.9/site-packages/rqdatac/client.py:241: UserWarning: Your account will be expired after  13 days. Please call us at 0755-22676337 to upgrade or purchase or renew your contract.
  warnings.warn("Your account will be expired after  {} days. "


In [ ]:
fund_data = fund.all_instruments(date=None)
fund_requirement = ['普通股票型', '偏股混合', '平衡混合', '灵活配置']
fund_list = []
for i in range(len(fund_data)):
    for j in fund_requirement:
        if (j in fund_data.symbol[i]) or (fund_data.fund_type[i] == 'Stock'):
            fund_list.append(fund_data.order_book_id[i])
fund_list = list(set(fund_list))
fund_data_needed = fund_data[fund_data.order_book_id.isin(fund_list)]

In [ ]:
def fund_get_indicator(fund_list, date='2023-07-14'):
    date_mid = datetime.datetime.strptime(date, '%Y-%m-%d')
    dateX = (date_mid - relativedelta(months=3)).strftime('%Y-%m-%d')
    # Ensure the dateX is a trading day
    trading_day_X = get_trading_dates(start_date=dateX, end_date=dateX)
    while not trading_day_X:  # If it's not a trading day, find the previous trading day
        dateX = str((datetime.datetime.strptime(dateX, '%Y-%m-%d') - relativedelta(days=1)).strftime('%Y-%m-%d'))
        trading_day_X = get_trading_dates(start_date=dateX, end_date=dateX)
    dateX = trading_day_X[0].strftime('%Y-%m-%d')

    performance = fund.get_indicators(fund_list, start_date=date, end_date=date,
                                      fields=['m3_return', 'm3_benchmark_return']).reset_index(drop=False)
    performance = performance.drop('datetime', axis=1).rename({'m3_return':'m3_return_y', 'm3_benchmark_return':'m3_benchmark_return_y'}, axis=1)

    indicators = fund.get_indicators(fund_list, start_date=dateX, end_date=dateX)
    indicators_m3 = indicators[[i for i in indicators.columns if 'm3' in i]].reset_index(drop=False)
    indicators_m3 = indicators_m3.drop(['m3_recovery_days', 'datetime', 'm3_return_a',
                                        'm3_excess', 'm3_excess_a', 'm3_excess_win'], axis=1).rename({'m3_return':'m3_return_x'}, axis=1)
    indicators_m3 = indicators_m3.loc[[j for j in indicators_m3.index if ~indicators_m3.loc[j].isna().any()]]
    df_returned = performance.merge(indicators_m3, on='order_book_id')
    return df_returned


In [ ]:
start_date = datetime.datetime.strptime('2010-07-14', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2023-07-14', '%Y-%m-%d')

seasons = {'winter': 0, 'spring': 3, 'summer': 6, 'autumn': 9} # Shifting months for each season
datasets = {}

for season, shift_month in seasons.items():
    # Generate all end dates for the 1-year periods within the 10 years, shifted for the season
    frequency = pd.date_range(start=start_date + pd.DateOffset(months=shift_month), end=end_date, freq='12M')
    concatenated_df = pd.DataFrame()
    for date in tqdm(frequency):
        # Convert the date to string format
        date_str = date.strftime('%Y-%m-%d')
        # Ensure the date is a trading day
        trading_day = get_trading_dates(start_date=date_str, end_date=date_str)
        while not trading_day:  # If it's not a trading day, find the previous trading day
            date = date - relativedelta(days=1)
            date_str = date.strftime('%Y-%m-%d')
            trading_day = get_trading_dates(start_date=date_str, end_date=date_str)
        # Call the function for each trading day and append the result to the dataframe
        df = fund_get_indicator(fund_list, date=trading_day[0].strftime('%Y-%m-%d'))
        concatenated_df = pd.concat([concatenated_df, df], axis=0)
    # Add the concatenated dataframe to the datasets
    datasets[season] = concatenated_df

In [ ]:
for i in seasons.keys():
    datasets[i].to_excel(f'factor_{i}_3m.xlsx')

In [ ]:
dateX = ('2023-07-17')
indicators = fund.get_indicators(fund_list, start_date=dateX, end_date=dateX)
indicators_m3 = indicators[[i for i in indicators.columns if 'm3' in i]].reset_index(drop=False)
indicators_m3 = indicators_m3.drop(['m3_recovery_days',    'datetime', 'm3_return_a',
                                        'm3_excess', 'm3_excess_a', 'm3_excess_win'],
                                       axis=1).rename({'m3_return': 'm3_return_x'}, axis=1)
indicators_m3 = indicators_m3.set_index('order_book_id')

In [19]:
start_date = datetime.datetime.strptime('2019-07-14', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2023-07-14', '%Y-%m-%d')
frequency = pd.date_range(start=start_date + pd.DateOffset(months=3), end=end_date, freq='3M')
concatenated_df = pd.DataFrame()
for date in tqdm(frequency):
    # Convert the date to string format
    date_str = date.strftime('%Y-%m-%d')
    # Ensure the date is a trading day
    trading_day = get_trading_dates(start_date=date_str, end_date=date_str)
    while not trading_day:  # If it's not a trading day, find the previous trading day
        date = date - relativedelta(days=1)
        date_str = date.strftime('%Y-%m-%d')
        trading_day = get_trading_dates(start_date=date_str, end_date=date_str)
    # Call the function for each trading day and append the result to the dataframe
    df = fund_get_indicator(fund_list, date=trading_day[0].strftime('%Y-%m-%d'))
    concatenated_df = pd.concat([concatenated_df, df], axis=0)
# Add the concatenated dataframe to the datasets
concatenated_df = concatenated_df.dropna()

100%|██████████| 15/15 [02:48<00:00, 11.24s/it]


In [20]:
reg = LinearRegression()
reg.fit(concatenated_df[concatenated_df.columns[3:]], concatenated_df['m3_return_y'])

LinearRegression()

In [ ]:
coef = {}
for i in range(reg.coef_.shape[0]):
    coef[concatenated_df.columns[3:][i]] = reg.coef_[i]
coef['intercept'] = reg.intercept_

In [ ]:
reg.score(concatenated_df[concatenated_df.columns[3:]], concatenated_df['m3_return_y'])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(concatenated_df[concatenated_df.columns[3:]], concatenated_df['m3_return_y'], test_size=0.30)
reg1 = RandomForestRegressor(n_estimators=120,max_leaf_nodes=50, max_depth=25)
reg1.fit(X_train, y_train)
reg1.score(X_test, y_test)

0.6306476734527382

In [49]:
concatenated_df.sort_values(by='m3_return_y',ascending=False)

,order_book_id,m3_return_y,m3_benchmark_return_y,m3_return_x,m3_benchmark_return,m3_stdev_a,m3_dev_downside_avg_a,m3_dev_downside_rf_a,m3_mdd,m3_excess_mdd,...,m3_beta_upside,m3_var,m3_alpha_a,m3_alpha_tstats,m3_beta,m3_sharpe_a,m3_inf_a,m3_sortino_a,m3_calmar_a,m3_timing_ratio
1774,006279,2.582863,0.048374,0.002040,0.016985,0.012912,0.001961,0.002745,0.003136,0.078905,...,-0.011135,-0.000185,-0.013949,-0.536058,-0.002653,-1.091671,-0.500960,-5.134283,-0.017835,-0.892616
90,000689,0.873070,0.056521,-0.100602,-0.010119,0.326851,0.230405,0.245511,0.213666,0.133564,...,1.773534,-0.039575,-0.339917,-0.897180,1.582320,-1.297589,-1.777844,-1.727492,-0.007877,1.192545
1610,005669,0.860799,-0.048647,-0.095898,-0.041720,0.334772,0.239189,0.253301,0.226556,0.134017,...,1.308414,-0.040401,-0.206516,-0.446011,1.072878,-1.192479,-0.977410,-1.576023,-0.006992,1.255272
1945,150050,0.745704,0.221195,0.235769,-0.022582,0.567617,0.436620,0.408551,0.249553,0.130678,...,1.815782,-0.042478,1.104092,2.324378,1.728281,1.748933,3.284408,2.429868,0.015786,1.102145
385,001404,0.668685,-0.048647,-0.079618,-0.041720,0.287440,0.209281,0.220869,0.191291,0.110818,...,0.983632,-0.035487,-0.188035,-0.427140,0.832436,-1.174307,-0.729444,-1.528250,-0.007002,1.141769
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,002051,-0.317406,-0.104881,-0.211836,-0.052169,0.320765,0.234598,0.265207,0.303646,0.277916,...,2.059294,-0.034346,-0.429550,-0.998848,2.070543,-2.782184,-2.676444,-3.365022,-0.011663,0.836579
389,001411,-0.317836,-0.104881,-0.211334,-0.052169,0.320994,0.235192,0.265709,0.303910,0.278289,...,2.054841,-0.034122,-0.426374,-0.991492,2.073362,-2.772261,-2.664660,-3.349081,-0.011619,0.832596
2325,012930,-0.324129,-0.119288,0.055349,-0.061658,0.249791,0.186527,0.179200,0.108896,0.089313,...,0.759791,-0.026035,0.447143,1.046659,0.885893,0.870651,2.191293,1.213623,0.007925,0.717628
2762,519615,-0.338581,-0.028756,0.037110,0.033942,0.061977,0.040720,0.036485,0.021848,0.055194,...,0.417432,-0.005546,0.081054,0.951327,0.382824,2.047355,0.085641,3.477827,0.023047,1.186304


In [52]:
reg1.predict(concatenated_df[concatenated_df.columns[3:]][concatenated_df.order_book_id == '000689'])

array([ 0.07174246,  0.06125853,  0.01972994,  0.22324137,  0.01416979,
        0.15931518,  0.01316222,  0.30274118,  0.01979303, -0.10661115,
       -0.1421772 ,  0.23455138, -0.11617977,  0.08957056, -0.06049203])

In [53]:
concatenated_df[concatenated_df.columns[3:]][concatenated_df.order_book_id == '000689']

,m3_return_x,m3_benchmark_return,m3_stdev_a,m3_dev_downside_avg_a,m3_dev_downside_rf_a,m3_mdd,m3_excess_mdd,m3_mdd_days,m3_max_drop,m3_max_drop_period,...,m3_beta_upside,m3_var,m3_alpha_a,m3_alpha_tstats,m3_beta,m3_sharpe_a,m3_inf_a,m3_sortino_a,m3_calmar_a,m3_timing_ratio
90,0.017259,0.016985,0.207026,0.158303,0.156460,0.044210,0.023127,19.0,-0.060914,5.0,...,1.414556,-0.017895,-0.011571,-0.099993,1.444468,0.330465,0.155421,0.437268,0.006141,0.920055
90,0.014127,0.054005,0.104109,0.071215,0.069601,0.042616,0.045870,6.0,-0.010805,6.0,...,0.989939,-0.010132,-0.168219,-1.765510,1.036863,0.409177,-3.448270,0.612049,0.003967,0.936626
90,0.038806,0.083999,0.106682,0.081749,0.077716,0.036012,0.042680,3.0,-0.026120,4.0,...,0.942713,-0.008645,-0.175315,-2.209150,1.038263,1.211404,-4.054817,1.662913,0.014241,0.831009
90,0.032568,0.032690,0.283958,0.222003,0.217436,0.145486,0.053389,12.0,-0.058429,4.0,...,1.223548,-0.035842,-0.014630,-0.075314,1.229212,0.530603,0.148394,0.692937,0.004110,0.984835
90,0.242114,0.202823,0.319493,0.235498,0.207605,0.101884,0.073097,9.0,-0.059898,4.0,...,1.941136,-0.027465,-0.456057,-1.936100,1.804385,2.984927,0.981145,4.593644,0.037144,1.114471
90,0.014739,0.029673,0.256373,0.186631,0.183829,0.096154,0.059837,25.0,-0.032885,5.0,...,1.523913,-0.027966,-0.113501,-0.505300,1.664758,0.289517,-0.268668,0.403769,0.003063,0.923282
90,0.328249,0.104899,0.411780,0.289220,0.248235,0.088317,0.079439,9.0,-0.060819,4.0,...,2.578460,-0.033975,0.267597,0.548438,2.411651,2.816353,2.516951,4.671857,0.052108,1.181858
90,-0.100602,-0.010119,0.326851,0.230405,0.245511,0.213666,0.133564,31.0,-0.048054,7.0,...,1.773534,-0.039575,-0.339917,-0.897180,1.582320,-1.297589,-1.777844,-1.727492,-0.007877,1.192545
90,0.852577,0.058436,0.400593,0.276603,0.196057,0.057828,0.064512,2.0,-0.046880,3.0,...,2.866217,-0.029465,2.141942,3.832930,2.121262,6.538039,7.597276,13.358805,0.179727,2.032801
90,0.198301,0.026626,0.408694,0.280629,0.250487,0.131776,0.109012,12.0,-0.053359,5.0,...,2.740887,-0.043882,0.572598,1.118646,2.586333,2.050532,2.312843,3.345640,0.025236,1.180721


In [61]:
dateX = '2018-01-31'
a = fund.get_indicators(fund_list, fields=['m1_return'], start_date=dateX, end_date=dateX)

In [62]:
a

,,m1_return
order_book_id,datetime,
000029,2018-01-31,0.027857
000030,2018-01-31,0.004887
000058,2018-01-31,0.004792
000062,2018-01-31,0.000763
000063,2018-01-31,-0.070853
...,...,...
750001,2018-01-31,0.034019
762001,2018-01-31,-0.007786
770001,2018-01-31,0.067962
